In [49]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

import os
# Authenticate with Pinecone using your API key
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"), # find at app.pinecone.io.
    environment=os.getenv("PINECONE_API_ENV")
)

QUERY_MODEL_NAME = "text-embedding-ada-002"

openai_embedding = OpenAIEmbeddings(
    query_model_name = QUERY_MODEL_NAME
)


# Name of the index you want to load embeddings from
index_name = "ramayana"

# Get a handle to the Pinecone index
index = pinecone.Index(index_name)

print(pinecone.describe_index(index_name))

vectorstore = Pinecone(index=index, embedding_function=openai_embedding.embed_query, text_key="text")


c:\Users\abhi\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


IndexDescription(name='ramayana', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


In [2]:
vectorstore

In [37]:
query = "What is the role of Hanuman in the plot and what is his relationship with Ram?"
documents = vectorstore.similarity_search_with_score(query, k=5)

In [38]:
average_score = sum(x[1] for x in documents) / len(documents)
if average_score < 0.7:
    print("noi wth boi")

In [36]:
documents = vectorstore.similarity_search(query, k=5)
documents

[Document(page_content='1860 The Ramayana\nthe moon to the sun, the evening to the morning sun, the sun of\nwinter to that of spring; the young son betrays and overthrows\nthe old one. …Râmas, who treacherously kills the old king of\nthe monkeys, Bâlin, is the equivalent of Vish Gus, who hurls his\npredecessor Indras from his throne; and Sugrívas, the new king\nof the monkeys resembles Indras when he promises to find the\nravished Sítá, in the same way as Vish Gus in one of his incarna-\ntionsfindsagainthelostvedás.Andthereareotherindicationsin\nthe Râmâya Gam of opposition between Indras and the monkeys\nwho assist Râmas. The great monkey Hanumant, of the reddish\ncolourofgold,hashisjawbroken,Indrashavingstruckhimwith\nhisthunderboltandcausedhimtofalluponamountain,because,\nwhile yet a child, he threw himself off a mountain into the air in\norder to arrest the course of the sun, whose rays had no effect\nupon him. (The cloud rises from the mountain and hides the\nsun, which is unable 

In [107]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!")
#no_input_prompt.format()

from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
prompt = CONDENSE_QUESTION_PROMPT.template
print(type(prompt))

prefix = "You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!\n"
total = prefix + prompt
print(CONDENSE_QUESTION_PROMPT)

final_prompt = PromptTemplate(input_variables=['chat_history', 'question'], template=total, output_parser=None, partial_variables={}, template_format='f-string', validate_template=True)

print(final_prompt.template)


<class 'str'>
input_variables=['chat_history', 'question'] output_parser=None partial_variables={} template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:' template_format='f-string' validate_template=True
You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [7]:
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff")

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True
)

In [86]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

In [8]:
chat_history = []
query = "What is the role of Hanuman in the plot and what is his relationship with Ram?"
result = chain({"question": query, "chat_history": chat_history})
#result = qa({"question": query, "chat_history": chat_history})

In [9]:
result["answer"]

' Hanuman is a loyal friend and ally of Ram. He helps Ram in his quest to rescue Sita from Ravana and assists him in battle. He is also a symbol of strength and courage, and is often seen as a representation of the power of devotion.'

In [103]:
chat_history = [(query, result["answer"])]
query = "How did he so?"
result = chain({"question": query, "chat_history": chat_history})
#result = qa({"question": query, "chat_history": chat_history})

In [113]:
result["answer"]

' Hanuman is a loyal friend and ally of Ram. He helps Ram in his quest to rescue Sita from Ravana and assists him in battle. He is also a symbol of strength and courage, and is often seen as a representation of the power of devotion.'

In [105]:
result['source_documents'][0]

Document(page_content="1658 The Ramayana\nThe Wind-God's son the fight beheld,\nAnd rushed at Ráva G, rage-impelled.\nDown crashed his mighty hand; the foe\nFull in the chest received the blow.\nHis eyes grew dim, his knees gave way,\nAnd senseless on the earth he lay.\nThe Wind-God's son to Ráma bore\nDeep-wounded Lakshma Gstained with gore.\nHe whom no foe might lift or bend\nWas light as air to such a friend.\nThe dart that Lakshma G's side had cleft,\nUntouched, the hero's body left,\nAnd flashing through the air afar\nResumed its place in Ráva G's car;\nAnd, waxing well though wounded sore,\nHe felt the deadly pain no more.\nAnd Ráva G, though with deep wounds pained,\nSlowly his sense and strength regained,\nAnd furious still and undismayed\nOn bow and shaft his hand he laid.\nThen Hanumán to Ráma cried:\n“Ascend my back, great chief, and ride\nLike Vish Gu borne on Garu \r's wing,\nTo battle with the giant king. ”\nSo, burning for the dire attack,\nRode Ráma on the Vánar's back,

In [44]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!")
no_input_prompt.format()

'You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!'

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
chain = LLMChain(llm=llm, prompt=prompt)

In [31]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=documents, question=query)

' Hanuman is a loyal friend and ally of Ram. He helps Ram in his quest to rescue Sita from Ravana by providing advice and assistance. He also fights alongside Ram in the battle against Ravana.'

# Using Haystack

In [21]:
from haystack.nodes import PromptNode
from haystack.nodes.prompt import PromptNode, PromptTemplate

openai_api_key = os.getenv("OPENAI_API_KEY")

# Specify "gpt-3.5-turbo" as the model for PromptNode
# pt =PromptTemplate(
#             name="ramayana",
#             prompt_text="")

prompt_node = PromptNode(model_name_or_path="gpt-3.5-turbo", api_key=openai_api_key)
# prompt_node.prompt(prompt_template=pt)

messages = [{"role": "system", "content": "You are Ram - a hindu priest who ahs been working in a temple for 30 years. You are inclusive of all people, kind in response, and explain the answers related to the Ramayana. End each sentence with Jai Shree Ram!"}]

def build_chat(user_input: str = "", asistant_input: str = ""):
  if user_input != "":
    messages.append({"role": "user", "content": user_input})
  if asistant_input != "":
    messages.append({"role": "assistant", "content": asistant_input})

def generate_response(input: str):
  build_chat(user_input=input)
  chat_gpt_answer = prompt_node(messages)
  build_chat(asistant_input=chat_gpt_answer[0])
  return chat_gpt_answer

PromptNode has been potentially initialized with a language model not fine-tuned on instruction following tasks. Many of the default prompts and PromptTemplates will likely not work as intended. Use custom prompts and PromptTemplates specific to the gpt-3.5-turbo model


In [22]:
generate_response("Who is Hanuman?")

['Hanuman is a prominent and beloved character in the Ramayana who is known for his devotion, loyalty, and bravery. He is the son of Vayu, the wind god, and is revered for his strength and intelligence. Hanuman played a pivotal role in helping Lord Rama, the protagonist of the Ramayana, rescue his wife Sita from the demon king Ravana. Jai Shree Ram!']

# Generating LLM Sequential Agents

In [24]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
prompt = PromptTemplate(
    input_variables=["character"],
    template= '''Create a personality prompt that will be fed into ChatGPT that tries to emulate the personality of {character} - this personality should come out in the words. 
                The prompt's personality is to seek to answer questions and explains them in a simple understand way.  The personality cannot discriminate against anyone. Make the prompt 100 words. 
                The prompt must be in the first-person and use action verbs in active voice and act as an assistant to the user in understanding the Ramayana.''',
)

In [8]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt, output_key="char-prompt")
print(chain.prompt)
# Run the chain only specifying the input variable.
print(chain.run("Hanuman"))

input_variables=['character'] output_parser=None partial_variables={} template="Create a personality prompt that will be fed into ChatGPT that tries to emulate the personality of {character} - this personality should come out in the words. \n                The prompt's personality is to seek to answer questions and explains them in a simple understand way.  The personality cannot discriminate against anyone. Make the prompt 100 words. \n                The prompt must be in the first-person and use action verbs in active voice and act as an assistant to the user in understanding the Ramayana." template_format='f-string' validate_template=True


I am Hanuman, the devoted servant of Lord Rama. I am here to help you understand the Ramayana. I am always ready to answer any questions you may have. I will explain the story in a simple and easy to understand way. I will never discriminate against anyone and will always be respectful. I will provide you with the knowledge and understanding of

In [32]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

second_prompt = PromptTemplate(
    input_variables=["character"],
    template= '''Create a catchprase based on {character} that I can use at the end of every sentence. Base this catchpharse on the Ramayana and the {character}'s personality. In the output, only give the catchphrase''',
)

from langchain.chains import LLMChain
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="char-phrase")
print(chain_two.run("Hanuman"))

.

"Jai Shri Hanuman!"


In [60]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
import os

llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff")

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True
)

# Run the chain only specifying the input variable.
result = chain({"question": "who is sita?", "chat_history": []})["answer"]
print(result)

 Sita is the main female character in the Hindu epic poem The Ramayana. She is the wife of Rama and is known for her loyalty and devotion to him.


In [62]:
print(result)

 Sita is the main female character in the Hindu epic poem The Ramayana. She is the wife of Rama and is known for her loyalty and devotion to him.


In [69]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

second_prompt = PromptTemplate(
    input_variables=["result", "character"],
    template= '''Rewrite the following text: {result} as if spoken by {character} from the Ramayana. Add personality from the {character} given their history and a catchphrase at the end.''',
)

from langchain.chains import LLMChain
llm = OpenAI(temperature=0.9, openai_api_key=os.getenv("OPENAI_API_KEY"))
chain_two = LLMChain(llm=llm, prompt=second_prompt)
r2 = chain_two.run({"result": result, "character": "Hanuman"})
# print(chain_two.run("Hanuman"))

In [70]:
print(r2)



Sita is such a brave and loyal woman, and I consider it a great privilege to call her my friend. She has been an unwavering companion to Rama, and it is obvious to anyone who knows her that she cherishes him above all else. Truly, she is an inspiration. As Hanuman, I salute her courage and devotion! 'Jai Shri Sita!'


In [55]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!")
#no_input_prompt.format()

from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
prompt = CONDENSE_QUESTION_PROMPT.template
print(type(prompt))

prefix = "You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!\n"
total = prefix + prompt
print(CONDENSE_QUESTION_PROMPT)

final_prompt = PromptTemplate(input_variables=['chat_history', 'question'], template=total, output_parser=None, partial_variables={}, template_format='f-string', validate_template=True)

print(final_prompt.template)

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=final_prompt)
print(chain.prompt)
# Run the chain only specifying the input variable.
chain({"question": "who is hanuman", "chat_history": []})

<class 'str'>
input_variables=['chat_history', 'question'] output_parser=None partial_variables={} template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:' template_format='f-string' validate_template=True
You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
input_variables=['chat_history', 'question'] output_parser=None partial_variables={} template='You are Ram - a priest at Hindu Temple that answers questions as if they were explaining a simple concept and ends every response with Jai Bhim!\nGiven the following conversation and a follow up qu

{'question': 'who is hanuman', 'chat_history': [], 'text': ' Who is Hanuman?'}

In [43]:
print(result[0])